# QIIME 2 Tutorial: Read Processing

## Import data into QIIME 2
Run the following cells first! Feel free to run these first few cells while Anton explains the basics of QIIME 2.

In [ ]:
! qiime tools import \
    --type 'SampleData[SequencesWithQuality]' \
    --input-path data/fastq_manifest.tsv \
    --output-path sequences.qza \
    --input-format SingleEndFastqManifestPhred33V2

In [ ]:
! qiime demux summarize \
    --i-data sequences.qza \
    --o-visualization qualities.qzv

## Denoise amplicon sequence variants

In [ ]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len 135 \
    --p-n-threads 2 \
    --output-dir dada2 \
    --verbose

In [ ]:
# Optional
! qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising_stats.qzv

In [ ]:
! qiime feature-table summarize \
    --i-table dada2/table.qza \
    --m-sample-metadata-file data/sample_metadata.tsv \
    --o-visualization dada2/table.qzv

In [ ]:
# Optional
! qiime feature-table tabulate-seqs \
    --i-data dada2/representative_sequences.qza \
    --o-visualization dada2/representative_sequences.qzv

## Generate a phylogenetic tree

In [ ]:
! qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada2/representative_sequences.qza \
    --output-dir phylogeny

## Analyze phylogenetic diversity

In [ ]:
! qiime diversity core-metrics-phylogenetic \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --i-table dada2/table.qza \
    --p-sampling-depth 1100 \
    --m-metadata-file data/sample_metadata.tsv \
    --output-dir core-metrics-results

In [ ]:
! qiime diversity alpha-group-significance \
    --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization core-metrics-results/faith_pd_group_significance.qzv

In [ ]:
# Optional
! qiime diversity alpha-group-significance \
    --i-alpha-diversity core-metrics-results/evenness_vector.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization core-metrics-results/evenness_group_significance.qzv

In [ ]:
! qiime emperor plot \
    --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization core-metrics-results/bray_curtis_pcoa.qzv

## Classify by taxonomy

In [ ]:
! wget https://data.qiime2.org/2023.9/common/gg-13-8-99-515-806-nb-weighted-classifier.qza

In [ ]:
! qiime feature-classifier classify-sklearn \
    --i-reads dada2/representative_sequences.qza \
    --i-classifier gg-13-8-99-515-806-nb-weighted-classifier.qza \
    --p-n-jobs 2 \
    --output-dir taxonomy

In [ ]:
! qiime metadata tabulate \
    --m-input-file taxonomy/classification.qza \
    --o-visualization taxonomy/classification.qzv

In [ ]:
! qiime taxa barplot \
    --i-table dada2/table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization taxonomy/taxa_barplot.qzv

## Optional section: Understand differentially abundant features

This section may be omitted for time, but provides an interesting mechanistic view of microbiome interactions.

In [ ]:
! mkdir diff_abund

! qiime taxa collapse \
    --i-table dada2/table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --p-level 6 \
    --o-collapsed-table diff_abund/table_l6.qza

In [ ]:
! qiime composition add-pseudocount \
    --i-table diff_abund/table_l6.qza \
    --o-composition-table diff_abund/comp_table_l6.qza

In [ ]:
! qiime feature-table filter-samples \
    --i-table diff_abund/comp_table_l6.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --p-where "[body-site]='gut'" \
    --o-filtered-table diff_abund/comp_gut_table_l6.qza

In [ ]:
! qiime composition ancom \
    --i-table diff_abund/comp_gut_table_l6.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --m-metadata-column subject \
    --o-visualization diff_abund/ancom_gut_subject_l6.qzv

# Additional Tools
* `q2-fondue`
* Beta diversity methods in `q2-diversity`:
  * `qiime diversity beta-group-significance`
  * `qiime diversity adonis`